In [16]:
import pandas as pd
from Constraint import Problem, Constraint
from Nature import attribute_nature
import copy
from constraint_relax import search_mfs
from item_extrator import data_path, dummy_data_path

In [17]:
experiment_constraint_data = pd.read_csv('./experiment_constraint_data.csv')

In [18]:
# read experiment constraints data
problem_list = list()
unique_constraint_no = experiment_constraint_data['constraint_no'].unique()
for no in unique_constraint_no:
    problem: Problem = Problem()
    constraint_list = list()
    for index, item in experiment_constraint_data[experiment_constraint_data['constraint_no'] == no].iterrows():
        constraint = None
        if item.get('name') in ['brand', 'cpu','gpu','os']:
            constraint: Constraint = Constraint(item.get('name'), item.get('value'), item.get('priority'),
                                                attribute_nature.get(item.get('name')))
        elif item.get('name') in ['storage', 'ram','price','weight']:
            constraint: Constraint = Constraint(item.get('name'), float(item.get('value')), item.get('priority'),
                                                attribute_nature.get(item.get('name')))
        elif item.get('name') in ['camera']:
            constraint: Constraint = Constraint(item.get('name'), bool(item.get('value')), item.get('priority'),
                                                attribute_nature.get(item.get('name')))
        if constraint != None:
            constraint_list.append(constraint)
    cl_set = set(constraint_list)
    problem.constraint_list = set(constraint_list)
    problem_list.append(problem)

In [23]:
def get_df_len(df):
    if df is None:
        return 0
    return (df.shape)[0]

sol_list = []
threshold_list = [0, 0.5, 1]
for problem in problem_list:
    prob_sol_list = []
    p1 = copy.deepcopy(problem) # original constraint
    prob_sol_list.append(p1.constraint_list)
    prob_sol_list.append(get_df_len(p1.solve()))

    for threshold in threshold_list:
        p = copy.deepcopy(problem)
        mfs = search_mfs(p.constraint_list, copy.copy(p.constraint_list), dummy_data_path)
        p.relax(mfs, threshold)
        prob_sol_list.append(p.constraint_list)
        prob_sol_list.append(get_df_len(p.solve()))
    sol_list.append(prob_sol_list)
result = pd.DataFrame(sol_list)

In [24]:
result

,0,1,2,3,4,5,6,7
0,"{storage: 512.0: 5: Nature.MORE, cpu: i7: 5: N...",0,"{storage: 512.0: 5: Nature.MORE, os: 10: 5: Na...",1,"{os: 10: 5: Nature.EQUAL, ram: 32: 3: Nature.M...",1,"{os: 10: 5: Nature.EQUAL, storage: 512.0: 5: N...",19
1,"{brand: microsoft: 4: Nature.EQUAL, storage: 1...",0,"{brand: microsoft: 4: Nature.EQUAL, price: 236...",0,"{price: 2368.59: 1: Nature.LESS, storage: : 2:...",0,"{brand: microsoft: 4: Nature.EQUAL, cpu: penti...",0
2,"{ram: 2.0: 1: Nature.MORE, storage: 1024.0: 3:...",0,"{ram: 2.0: 1: Nature.MORE, brand: : 3: Nature....",0,"{os: 10|os: 3: Nature.EQUAL, brand: lenovo|msi...",0,"{storage: 1024.0: 3: Nature.MORE, ram: 2.0: 1:...",0
3,"{cpu: ryzen 3: 2: Nature.EQUAL, ram: 16.0: 5: ...",0,"{camera: True: 4: Nature.EQUAL, ram: 16.0: 5: ...",6,"{camera: True: 4: Nature.EQUAL, ram: 16.0: 5: ...",26,"{cpu: : 2: Nature.EQUAL, ram: 16.0: 5: Nature....",26
4,"{price: 4965.89: 2: Nature.LESS, brand: asus|h...",10,"{price: 4965.89: 2: Nature.LESS, camera: True:...",11,"{storage: : 1: Nature.MORE, brand: asus|huawei...",12,{brand: asus|huawei|microsoft|msi: 3: Nature.E...,12
...,...,...,...,...,...,...,...,...
132,"{os: 11: 3: Nature.EQUAL, camera: True: 4: Nat...",5,"{price: 2911.6: 2: Nature.LESS, os: 11: 3: Nat...",9,"{brand: : 1: Nature.EQUAL, os: 11: 3: Nature.E...",9,"{storage: 256.0: 3: Nature.MORE, brand: : 1: N...",9
133,"{storage: 256.0: 2: Nature.MORE, camera: True:...",5,"{weight: 1.44: 4: Nature.LESS, camera: True: 3...",24,"{cpu: i7|i5: 2: Nature.EQUAL, weight: 1.44: 4:...",25,"{camera: True: 3: Nature.EQUAL, weight: 1.44: ...",25
134,"{storage: 64.0: 5: Nature.MORE, camera: True: ...",23,"{camera: : 3: Nature.EQUAL, ram: 8.0: 3: Natur...",25,"{weight: 1.45: 5: Nature.LESS, camera: : 3: Na...",25,"{brand: : 3: Nature.EQUAL, weight: 1.45: 5: Na...",40
135,"{price: 7399.0: 2: Nature.LESS, cpu: i5: 1: Na...",0,"{cpu: ryzen 3|i5|ryzen 5: 1: Nature.EQUAL, bra...",0,"{cpu: : 1: Nature.EQUAL, storage: 256.0: 2: Na...",0,"{brand: : 1: Nature.EQUAL, price: 7399.0: 2: N...",0


In [28]:
pd.value_counts(result.iloc[:, 1] == 0)

True     82
False    55
Name: 1, dtype: int64

In [25]:
valid = ((result.iloc[:, 1] <= result.iloc[:, 3]) & (result.iloc[:, 3] <= result.iloc[:, 5]) & (result.iloc[:, 5] <= result.iloc[:, 7]))
valid_count = pd.value_counts(valid)
print(valid_count)

True     119
False     18
dtype: int64
